# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import torch

In [2]:
from src.vfm.connection import Connection
from src.vfm.preprocessor import Preprocessor
import os
import sys
import pandas as pd

In [2]:
connection = Connection()

# Add the parent directory to the sys.path list
# sys.path.insert(0, os.path.abspath('..'))

# Get client
client = connection.get_client()


In [3]:
well = 'W06'
# df_raw = connection.get_data(client=client, well=well, start=pd.Timestamp("2019-01-01"), end=pd.Timestamp("2022-04-18"))
df_raw = connection.get_data(client=client, well=well)
print(df_raw.shape)

(3244607, 12)


In [4]:
preprocessor = Preprocessor(df_raw=df_raw)
df = preprocessor.preprocess()
print(df.shape)

AssertionError: Choke position values greater than 1% found.

In [8]:
from sklearn.model_selection import train_test_split

# Select all independent variables.
X = df[['dhp', 'dht', 'whp', 'wht', 'choke', 'dcp']]
# Select only the target variable.
Y = df[['qo_mpfm', 'qg_mpfm', 'qw_mpfm']]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=65)

In [9]:
# Training data
print(f'Train data records  {X_train.shape}')

# Testing data
print(f'Test data records {X_test.shape}')

Train data records  (68, 6)
Test data records (18, 6)


# Feed Forward Neural Network Model - Well 06

In [10]:
from src.vfm.model.nn.feed_forward import FeedForwardNeuralNetworkEvaluator
ff_nn = FeedForwardNeuralNetworkEvaluator(X_train, Y_train, X_test, Y_test)
ff_nn.train()

c:\Users\perer\AppData\Local\pypoetry\Cache\virtualenvs\vfm-Whcif8Lq-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-12-03 03:09:49,795] A new study created in memory with name: no-name-fd889946-446b-4d64-8ee8-5c735d2a4443
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
You are using a CUDA device ('NVIDIA GeForce RTX 3050 6GB Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


num_hidden_layers = 18, num_hidden_features = 61, learning_rate = 4.5209425954525334e-05, num_epochs = 150


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                                          | Type       | Params | Mode  | FLOPs
---------------------------------------------------------------------------------------------
0 | _FeedForwardNeuralNetworkModel__criterion     | MSELoss    | 0      | train | 0    
1 | _FeedForwardNeuralNetworkModel__fc1           | Linear     | 427    | train | 0    
2 | _FeedForwardNeuralNetworkModel__hidden_layers | ModuleList | 68.1 K | train | 0    
3 | _FeedForwardNeuralNetworkModel__fc3           | Linear     | 186    | train | 0    
---------------------------------------------------------------------------------------------
68.7 K    Trainable params
0         Non-trainable params
68.7 K    Total params
0.275     Total estimated model params size (MB)
22        Modules in train mode
0         Modules in eval mode
0         Total Flops
c:\Users\perer\AppData\Local\pypoetry\Cache\virtualenvs\vfm-Whcif8Lq-py3.11\Lib\site-packages\pytorch_lightning\tr

Epoch 47:   0%|          | 0/2 [00:00<?, ?it/s, v_num=0]        


Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

c:\Users\perer\AppData\Local\pypoetry\Cache\virtualenvs\vfm-Whcif8Lq-py3.11\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
ff_nn.test()

Best model :
num_hidden_layers = 19, num_hidden_features = 58, learning_rate = 1.4743924513056339e-05, num_epochs = 112


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0:  22%|██▏       | 431/1998 [00:00<00:02, 559.04it/s]

TypeError: 'NoneType' object is not subscriptable

In [ ]:
ff_nn.results()